In [4]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.py
!pip install git+https://github.com/deepmind/dm-haiku
!pip install DotMap

--2020-09-05 13:02:04--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3956 (3.9K) [text/plain]
Saving to: ‘trust_ncg.py’

trust_ncg.py        100%[===================>]   3.86K  --.-KB/s    in 0s      

2020-09-05 13:02:04 (50.6 MB/s) - ‘trust_ncg.py’ saved [3956/3956]

--2020-09-05 13:02:04--  https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5789 (5.7K) [text/plain]
Saving to: ‘che_tools.

In [5]:
import jax.numpy as jnp
import jax
import che_tools as che
import haiku as hk
from dotmap import DotMap
from trust_ncg import minimize, minimize_hk

In [6]:
p=che.Props(['Isopropanol', 'Water'])


In [11]:

class Comp(hk.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

    def x(self):
        q = hk.get_parameter('q', shape=[p.N_comps-1],init=jnp.zeros)
        return p.qtox(q)

    def AvgMw(self):
        return jnp.vdot(self.x(), p.Mw)

class LiquidPhase(Comp):
    def __init__(self, name=None):
        super().__init__(name=name)

    def bubbleP(self):
        T = hk.get_parameter('T', shape=(1,), init=jnp.zeros)
        return jnp.vdot(self.x(), p.Pvap(T))

def bubbleT_f():
    liquid = LiquidPhase(name='liquid')
    return (liquid.bubbleP()-101325)**2

In [12]:

model = hk.transform(bubbleT_f)

In [13]:
c = DotMap()
c.liquid.q=p.xtoq(jnp.array([0.4,0.6]))
v = DotMap()
v.liquid.T = jnp.array([300])

In [14]:
min_v, min_f = minimize_hk(model, v, c, verbose=True)

0:0, f: 9307906198.914825
x: [301.]
grad: [-55494607.42191157]
dx: [1.]

1:0, f: 9251050985.632477
x: [303.]
grad: [-58232012.33119455]
dx: [2.]

2:0, f: 9128883848.753511
x: [307.]
grad: [-64000905.03172091]
dx: [4.]

3:0, f: 8847947274.855974
x: [315.]
grad: [-76734512.2650342]
dx: [8.]

4:0, f: 8117434091.349681
x: [331.]
grad: [-1.0690178e+08]
dx: [16.]

5:0, f: 5838926798.228783
x: [363.]
grad: [-1.78707684e+08]
dx: [32.]

6:0, f: 32525889.423041478
x: [364.66795424]
grad: [-41015635.4610771]
dx: [1.66795424]

7:1, f: 203907.86150768708
x: [364.54909445]
grad: [3418591.590076]
dx: [-0.11885979]

8:0, f: 6.0457246971041725
x: [364.54844253]
grad: [18547.0482558]
dx: [-0.00065192]

9:0, f: 5.418302098580751e-09
x: [364.54844251]
grad: [0.55523081]
dx: [-1.95172962e-08]

Final results:
f: 5.418302098580751e-09
x: [364.54844251]
grad: [0.55523081]
dx: [-1.95172962e-08]



In [15]:
DotMap(min_v).liquid.T

DeviceArray([364.54844251], dtype=float64)